In [ ]:
import pandas as pd
import numpy as np 
from pulp import *
import random
import itertools

In [ ]:
# list of drivers
drivers = ['W' + str(i) for i in range(1,11)]
# list of routes
routes = ['R' + str(i) for i in range(1,11)]
# list of route types
types = ['a','b','c','d']

# list of driver preferences
driver_preference = [random.choice(types) for i in range(1,11)]
# list of route types
route_type = [random.choice(types) for i in range(1,11)]
# list of potential start times
times = list(pd.date_range('07:00','09:00',freq='5min'))
# route start times
route_start_times = [random.choice(times) for i in range(1,11)]
# driver start times
driver_start_times = [random.choice(times) for i in range(1,11)]


In [ ]:
# create list of every combination of route and driver
combinations = list(itertools.product(routes,drivers))

# set up problem
allocation_model = LpProblem('Allocation', LpMaximize)
# create binary decision variables
var = LpVariable.dicts('VAR', combinations, lowBound=0, upBound=1, cat = LpInteger)


In [ ]:
# create preference matrix
preferences = np.zeros(len(combinations)).reshape(len(routes),len(drivers))
# iterate through every route
for i in range(len(routes)):
    for j in range(len(drivers)):
        # if preference does not match route type
        if route_type[i] != driver_preference[j]:
            preferences[i,j] = 1
        else:
            # if preferences match
            preferences[i,j] = 2


In [ ]:
difference = np.zeros(len(combinations)).reshape(len(routes),len(drivers))
# iterate through every route
for i in range(len(routes)):
    for j in range(len(drivers)):
        # compute difference in start time
        diff = abs(route_start_times[i] - driver_start_times[j])
        # convert from date time to minutes
        mins = diff.total_seconds()/60
        
        # give score between 0 and 1
        if mins < 10:
            difference[i,j] = 1
        elif mins < 20:
            difference[i,j] = 0.75
        elif mins < 30:
            difference[i,j] = 0.5
        elif mins < 45:
            difference[i,j] = 0.25
        elif mins < 60:
            difference[i,j] = 0.1
        else:
            difference[i,j] = -100
        

In [ ]:
def obj_func(comb):
   # seperate route and driver
    route, driver = comb
    # get index of route
    i = routes.index(route)
    # get index of driver
    j = drivers.index(driver)
    
    return difference[i,j] * preferences[i,j]


# add objective function to model
allocation_model += lpSum([obj_func(comb) * var[comb] for comb in combinations])


In [ ]:
# constraint 1 - a route can only appear once in the final solution.
for route in routes:
    allocation_model += lpSum([var[comb] for comb in combinations if route in comb]) <= 1

# constraint 2 - a driver can only appear once in the final solution.
for driver in drivers:
    allocation_model += lpSum([var[comb] for comb in combinations if driver in comb]) <= 1

In [ ]:
allocation_model.solve()

for comb in combinations:
    if var[comb].varValue() == 1:
        # get index of route
        i = routes.index(route)
        # get index of driver
        j = drivers.index(driver)
        # get information for route and driver
        route = (comb[0],route_start_times[i].strftime('%H:%M'),route_type[i])
        driver = (comb[1], driver_start_times[j].strftime('%H:%M'),driver_preference[j])
        # print solution
        print(route,driver)
        
        